![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertForSequenceClassification.ipynb)

## Import ONNX BertForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `BertForSequenceClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for token classification via `BertForSequenceClassification` or `TFBertForSequenceClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFBertForSequenceClassification](https://huggingface.co/transformers/model_doc/bert.html#tfbertforsequenceclassification)
- Some [example models](https://huggingface.co/models?filter=bert&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.51.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [finiteautomata/beto-sentiment-analysis](https://huggingface.co/finiteautomata/beto-sentiment-analysis) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the BERT model, we also need to save the `BertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import BertTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

('onnx_models/nlptown/bert-base-multilingual-uncased-sentiment/tokenizer_config.json',
 'onnx_models/nlptown/bert-base-multilingual-uncased-sentiment/special_tokens_map.json',
 'onnx_models/nlptown/bert-base-multilingual-uncased-sentiment/vocab.txt',
 'onnx_models/nlptown/bert-base-multilingual-uncased-sentiment/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 654916
-rw-r--r-- 1 root root      1081 Jun  9 21:05 config.json
-rw-r--r-- 1 root root 669743468 Jun  9 21:05 model.onnx
-rw-r--r-- 1 root root       125 Jun  9 21:05 special_tokens_map.json
-rw-r--r-- 1 root root      1290 Jun  9 21:05 tokenizer_config.json
-rw-r--r-- 1 root root    871891 Jun  9 21:05 vocab.txt


- We need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir -p {EXPORT_PATH}/assets

labels = ort_model.config.id2label
sorted_labels = [label for _, label in sorted(labels.items())]

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(sorted_labels))

!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [5]:
!ls -lR {EXPORT_PATH}

onnx_models/nlptown/bert-base-multilingual-uncased-sentiment:
total 654068
drwxr-xr-x 2 root root      4096 Jun  9 21:06 assets
-rw-r--r-- 1 root root      1081 Jun  9 21:05 config.json
-rw-r--r-- 1 root root 669743468 Jun  9 21:05 model.onnx
-rw-r--r-- 1 root root       125 Jun  9 21:05 special_tokens_map.json
-rw-r--r-- 1 root root      1290 Jun  9 21:05 tokenizer_config.json

onnx_models/nlptown/bert-base-multilingual-uncased-sentiment/assets:
total 856
-rw-r--r-- 1 root root     38 Jun  9 21:06 labels.txt
-rw-r--r-- 1 root root 871891 Jun  9 21:05 vocab.txt


In [6]:
!cat /content/onnx_models/nlptown/bert-base-multilingual-uncased-sentiment/assets/labels.txt

1 star
2 stars
3 stars
4 stars
5 stars

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save BertForSequenceClassification in Spark NLP


Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 46.3 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [9]:
from sparknlp.annotator import BertForSequenceClassification

sequenceClassifier = BertForSequenceClassification\
  .loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your BertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 654160
-rw-r--r-- 1 root root 669845797 Jun  9 21:09 bert_classification_onnx
drwxr-xr-x 4 root root      4096 Jun  9 21:09 fields
drwxr-xr-x 2 root root      4096 Jun  9 21:09 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForSequenceClassification model 😊

In [13]:
sequenceClassifier_loaded = BertForSequenceClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
sequenceClassifier_loaded.getClasses()

['4 stars', '3 stars', '1 star', '2 stars', '5 stars']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [19]:
example = spark.createDataFrame([
    ["Absolutely terrible. The product broke after one day.", "English", "1 star"],
    ["Matige kwaliteit, ik had meer verwacht.", "Dutch", "2 stars"],
    ["Ist in Ordnung, aber nichts Besonderes.", "German", "3 stars"],
    ["Bon produit, je suis satisfait.", "French", "4 stars"],
    ["¡Fantástico! Totalmente recomendado.", "Spanish", "5 stars"],
    ["Qualità scarsa, non soddisfatto.", "Italian", "2 stars"],
], ["text", "language", "expected_rating"])

example.show(truncate=False)

+-----------------------------------------------------+--------+---------------+
|text                                                 |language|expected_rating|
+-----------------------------------------------------+--------+---------------+
|Absolutely terrible. The product broke after one day.|English |1 star         |
|Matige kwaliteit, ik had meer verwacht.              |Dutch   |2 stars        |
|Ist in Ordnung, aber nichts Besonderes.              |German  |3 stars        |
|Bon produit, je suis satisfait.                      |French  |4 stars        |
|¡Fantástico! Totalmente recomendado.                 |Spanish |5 stars        |
|Qualità scarsa, non soddisfatto.                     |Italian |2 stars        |
+-----------------------------------------------------+--------+---------------+



In [23]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.sql.functions import col

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

result = pipeline.fit(example).transform(example)
result.withColumn("predicted_rating", col("class.result")[0]) \
    .select("text", "language", "expected_rating", "predicted_rating") \
    .show(truncate=False)

+-----------------------------------------------------+--------+---------------+----------------+
|text                                                 |language|expected_rating|predicted_rating|
+-----------------------------------------------------+--------+---------------+----------------+
|Absolutely terrible. The product broke after one day.|English |1 star         |1 star          |
|Matige kwaliteit, ik had meer verwacht.              |Dutch   |2 stars        |2 stars         |
|Ist in Ordnung, aber nichts Besonderes.              |German  |3 stars        |3 stars         |
|Bon produit, je suis satisfait.                      |French  |4 stars        |4 stars         |
|¡Fantástico! Totalmente recomendado.                 |Spanish |5 stars        |5 stars         |
|Qualità scarsa, non soddisfatto.                     |Italian |2 stars        |2 stars         |
+-----------------------------------------------------+--------+---------------+----------------+



That's it! You can now go wild and use hundreds of `BertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
